In [17]:
import os

from azureml.core import Environment, Model, Workspace
from azureml.core.model import InferenceConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core.webservice import AciWebservice
from azureml.core.compute import ComputeTarget

from dotenv import load_dotenv

load_dotenv()
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP")
AZURE_WORKSPACE_NAME = os.getenv("AZURE_WORKSPACE_NAME")


In [18]:
experiment_name = "oc-p8-experiment-1"
model_name = "deeplab_v3plus_128_augment"

# connect to your workspace
ws = Workspace(
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group=AZURE_RESOURCE_GROUP,
    workspace_name=AZURE_WORKSPACE_NAME,
)


model = Model(ws, model_name)

env_name = experiment_name + "-predict"
try:
    env = Environment.get(workspace=ws, name=env_name)
except:
    env = Environment.from_conda_specification(
        name=env_name, file_path="predict/conda_dependencies.yml"
    )
    env.register(workspace=ws)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=4,
)

inference_config = InferenceConfig(
    entry_script="predict/predict.py", environment=env
)

service = Model.deploy(
    workspace=ws,
    name=model_name.replace('_', '-'),
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-04 11:57:48+01:00 Creating Container Registry if not exists.
2022-03-04 11:57:50+01:00 Use the existing image.
2022-03-04 11:57:50+01:00 Generating deployment configuration.
2022-03-04 11:57:53+01:00 Submitting deployment to compute.
2022-03-04 11:57:58+01:00 Checking the status of deployment deeplab-v3plus-128-augment..
2022-03-04 11:59:52+01:00 Checking the status of inference endpoint deeplab-v3plus-128-augment.
Succeeded
ACI service creation operation finished, operation "Succeeded"
